In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [1]:
import pandas as pd
from pathlib import Path
import pyarrow.parquet as pq

month = 1
year = 2023
path = Path("..") / "data" / "processed" / f"rides_{year}_{month:02}.parquet"

table = pq.read_table(path)
rides = table.to_pandas()
rides.head()

,pickup_datetime,pickup_location_id
0,2023-01-01 00:32:10,161
1,2023-01-01 00:55:08,43
2,2023-01-01 00:25:04,48
3,2023-01-01 00:03:48,138
4,2023-01-01 00:10:29,107


In [3]:
rides.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2993140 entries, 0 to 2993139
Data columns (total 2 columns):
 #   Column              Dtype         
---  ------              -----         
 0   pickup_datetime     datetime64[us]
 1   pickup_location_id  int64         
dtypes: datetime64[us](1), int64(1)
memory usage: 45.7 MB


In [4]:
rides["pickup_hour"] = rides["pickup_datetime"].dt.floor("h")

In [5]:
rides.head()

,pickup_datetime,pickup_location_id,pickup_hour
0,2023-01-01 00:32:10,161,2023-01-01
1,2023-01-01 00:55:08,43,2023-01-01
2,2023-01-01 00:25:04,48,2023-01-01
3,2023-01-01 00:03:48,138,2023-01-01
4,2023-01-01 00:10:29,107,2023-01-01


In [6]:
rides.tail()

,pickup_datetime,pickup_location_id,pickup_hour
2993135,2023-01-31 23:58:34,107,2023-01-31 23:00:00
2993136,2023-01-31 23:31:09,112,2023-01-31 23:00:00
2993137,2023-01-31 23:01:05,114,2023-01-31 23:00:00
2993138,2023-01-31 23:40:00,230,2023-01-31 23:00:00
2993139,2023-01-31 23:07:32,262,2023-01-31 23:00:00


In [7]:
agg_rides = rides.groupby(["pickup_hour", "pickup_location_id"]).size().reset_index()
agg_rides.rename(columns={0: "rides"}, inplace=True)
agg_rides.tail()

,pickup_hour,pickup_location_id,rides
69772,2023-01-31 23:00:00,249,137
69773,2023-01-31 23:00:00,256,2
69774,2023-01-31 23:00:00,261,5
69775,2023-01-31 23:00:00,262,11
69776,2023-01-31 23:00:00,263,41


In [8]:
def fill_missing_rides_full_range(df, hour_col, location_col, rides_col):
    """
    Fills in missing rides for all hours in the range and all unique locations.

    Parameters:
    - df: DataFrame with columns [hour_col, location_col, rides_col]
    - hour_col: Name of the column containing hourly timestamps
    - location_col: Name of the column containing location IDs
    - rides_col: Name of the column containing ride counts

    Returns:
    - DataFrame with missing hours and locations filled in with 0 rides
    """
    # Ensure the hour column is in datetime format
    df[hour_col] = pd.to_datetime(df[hour_col])

    # Get the full range of hours (from min to max) with hourly frequency
    full_hours = pd.date_range(
        start=df[hour_col].min(),
        end=df[hour_col].max(),
        freq="h"
    )

    # Get all unique location IDs
    all_locations = df[location_col].unique()

    # Create a DataFrame with all combinations of hours and locations
    full_combinations = pd.DataFrame(
        [(hour, location) for hour in full_hours for location in all_locations],
        columns=[hour_col, location_col]
    )

    # Merge the original DataFrame with the full combinations DataFrame
    merged_df = pd.merge(full_combinations, df, on=[hour_col, location_col], how='left')

    # Fill missing rides with 0
    merged_df[rides_col] = merged_df[rides_col].fillna(0).astype(int)

    return merged_df

In [9]:
agg_rides

,pickup_hour,pickup_location_id,rides
0,2023-01-01 00:00:00,4,19
1,2023-01-01 00:00:00,7,3
2,2023-01-01 00:00:00,12,1
3,2023-01-01 00:00:00,13,14
4,2023-01-01 00:00:00,24,20
...,...,...,...
69772,2023-01-31 23:00:00,249,137
69773,2023-01-31 23:00:00,256,2
69774,2023-01-31 23:00:00,261,5
69775,2023-01-31 23:00:00,262,11


In [10]:
hour_col = "pickup_hour"
location_col = "pickup_location_id"
rides_col = "rides"
agg_data_filled = fill_missing_rides_full_range(agg_rides, hour_col, location_col, rides_col).sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

In [11]:
agg_data_filled

,pickup_hour,pickup_location_id,rides
0,2023-01-01 00:00:00,2,0
1,2023-01-01 01:00:00,2,0
2,2023-01-01 02:00:00,2,0
3,2023-01-01 03:00:00,2,0
4,2023-01-01 04:00:00,2,0
...,...,...,...
188971,2023-01-31 19:00:00,263,110
188972,2023-01-31 20:00:00,263,99
188973,2023-01-31 21:00:00,263,95
188974,2023-01-31 22:00:00,263,62


In [12]:
pd.set_option("display.max_rows", 2000)

In [13]:
agg_data_filled

,pickup_hour,pickup_location_id,rides
0,2023-01-01 00:00:00,2,0
1,2023-01-01 01:00:00,2,0
2,2023-01-01 02:00:00,2,0
3,2023-01-01 03:00:00,2,0
4,2023-01-01 04:00:00,2,0
...,...,...,...
188971,2023-01-31 19:00:00,263,110
188972,2023-01-31 20:00:00,263,99
188973,2023-01-31 21:00:00,263,95
188974,2023-01-31 22:00:00,263,62


In [14]:
rides[rides["pickup_location_id"]==2]

,pickup_datetime,pickup_location_id,pickup_hour
2622187,2023-01-28 17:03:38,2,2023-01-28 17:00:00


In [15]:
from typing import Optional, List
import plotly.express as px
import pandas as pd

def plot_rides(
    rides: pd.DataFrame,
    locations: Optional[List[int]] = None
):

    rides_to_plot = rides[rides.pickup_location_id.isin(locations)] if locations else rides

    fig = px.line(
        rides_to_plot,
        x="pickup_hour",
        y="rides",
        color="pickup_location_id",
        template="none"
    )

    fig.show()

In [16]:
plot_rides(agg_data_filled, [43,46])

In [17]:
plot_rides(agg_data_filled,locations=[11,21])

In [18]:
agg_data_filled['pickup_location_id'].unique()

array([  2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,
        15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  28,
        29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
        42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,
        55,  56,  57,  58,  60,  61,  62,  63,  64,  65,  66,  67,  68,
        69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,
        82,  83,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,
        96,  97,  98,  99, 100, 101, 102, 106, 107, 108, 109, 111, 112,
       113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
       126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138,
       139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151,
       152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164,
       165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 177, 178,
       179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 19

In [19]:
locations = pd.read_csv("D:/nyc_taxi/taxi_zone_lookup.csv")

In [20]:
locations.drop(columns=["service_zone"], inplace=True)

In [21]:
locations.drop(columns=["Borough"], inplace=True)

In [22]:
locations.head()

,LocationID,Zone
0,1,Newark Airport
1,2,Jamaica Bay
2,3,Allerton/Pelham Gardens
3,4,Alphabet City
4,5,Arden Heights


In [23]:
# agg_data_filled = agg_data_filled.merge(locations, left_on='pickup_location_id', right_on='LocationID', how='left')
# agg_data_filled.head()

In [24]:
agg_data_filled

,pickup_hour,pickup_location_id,rides
0,2023-01-01 00:00:00,2,0
1,2023-01-01 01:00:00,2,0
2,2023-01-01 02:00:00,2,0
3,2023-01-01 03:00:00,2,0
4,2023-01-01 04:00:00,2,0
...,...,...,...
188971,2023-01-31 19:00:00,263,110
188972,2023-01-31 20:00:00,263,99
188973,2023-01-31 21:00:00,263,95
188974,2023-01-31 22:00:00,263,62


In [25]:
agg_data_filled = agg_data_filled.merge(locations[['LocationID', 'Zone']], left_on='pickup_location_id', right_on='LocationID', how='left')
agg_data_filled.head()

,pickup_hour,pickup_location_id,rides,LocationID,Zone
0,2023-01-01 00:00:00,2,0,2,Jamaica Bay
1,2023-01-01 01:00:00,2,0,2,Jamaica Bay
2,2023-01-01 02:00:00,2,0,2,Jamaica Bay
3,2023-01-01 03:00:00,2,0,2,Jamaica Bay
4,2023-01-01 04:00:00,2,0,2,Jamaica Bay


In [26]:
from typing import Optional, List
import plotly.express as px
import pandas as pd

def plot_rides(
    rides: pd.DataFrame,
    zones: Optional[List[int]] = None
):

    rides_to_plot = rides[rides.Zone.isin(zones)] if zones else rides

    fig = px.line(
        rides_to_plot,
        x="pickup_hour",
        y="rides",
        color="Zone",
        template="none"
    )

    fig.show()

In [27]:
plot_rides(agg_data_filled, ["JFK Airport"])

In [28]:
month=1
year=2023
path = Path("..") / "data" / "processed" / f"ts_data_{year}_{month:02}.parquet"

agg_data_filled.to_parquet(path)